# Reader Monad

## Outline

* Incentive for the Reader monad

* The Reader monad
  - Definition of the Reader monad

  - Helper functions

  - MonadReader type class

  - Simple example
  
* Reader monad examples

In this lesson, we will learn about the Reader monad type and how you can use it. First we present the reason why introducing the Reader monad. Then we show the Reader monad definition and helper functions. We also learn a new ways how to create a type class by looking at the `MonadReader` type class. In the end we show some code examples that use the Reader monad.

## Incentive for Reader monad

Imagine we have a global environment which is represented by a variable that holds multiple data as e.g. a record syntax variable. 

If we have several functions that need to use part of that data and they call each other, we need to pass that variable from one to another.

Below is an example where we read 4 variables from the operating system environment and store them into a variable of type `Environment`. 

We process this data in 4 function calls and the last function writes the processed data to a configuration file.

In [ ]:
import Data.Maybe (fromMaybe)
import System.Environment (lookupEnv)
import qualified Data.Map as Map
import Maybes (fromJust)

type Key = String
type Content = String
newtype Environment = Environment
    { parameters :: Map.Map Key Content }

loadEnv :: IO Environment
loadEnv = do
  p1 <- lookupEnv "PATH"
  p2 <- lookupEnv "SHELL"
  p3 <- lookupEnv "HOSTNAME"
  p4 <- lookupEnv "HOME"
  return $ Environment $ Map.fromList 
    [ ("PATH", fromMaybe "" p1)
    , ("SHELL", fromMaybe "" p2)
    , ("HOSTNAME", fromMaybe "" p3)
    , ("HOME", fromMaybe "" p4)]

func1 :: Environment -> (Content, FilePath)
func1 env = func2 env content
    where content = "System path includes: \n" ++ parameter ++ "\n\n"
          parameter = fromJust $ Map.lookup "PATH" $ parameters env

func2 :: Environment -> Content -> (Content, FilePath)
func2 env content = func3 env updatedContent
    where updatedContent = content ++ "Shell used is: \n" ++ parameter ++ "\n\n"
          parameter = fromJust $ Map.lookup "SHELL" $ parameters env

func3 :: Environment -> Content -> (Content, FilePath)
func3 env content = func4 env updatedContent
    where updatedContent = content ++ "Hostname is: \n" ++ parameter ++ "\n\n"
          parameter = fromJust $ Map.lookup "HOSTNAME" $ parameters env

func4 :: Environment -> Content -> (Content, FilePath)
func4 env content = (updatedContent, filePath)
    where updatedContent = content ++ "Current home directory is: \n" ++ parameter ++ "\n\n"
          filePath = parameter ++ "/conf.txt"
          parameter = fromJust $ Map.lookup "HOME" $ parameters env

We can now write the main execution of our program where we read in the environment and process the data with our 4 functions.

In [ ]:
main :: IO ()
main = do
  env <- loadEnv
  let (updatedContent, filePath) = func1 env

  print "Writing configuration to file: conf.txt"
  writeFile filePath updatedContent

main

This is a simple example, but as complexity of programs increases you have to pass around a lot of data. 

For this reason the **Reader monad** was created which can store data in its environment variable.  

The data in the environment variable can then be passed between different Reader monads in the background.

## The Reader monad

### Definition of the Reader monad

The Haskell definition of the `Reader` type is defined in terms of the `ReaderT` monad transformer:
```haskell
type Reader r = ReaderT r Identity
```

It is contained in the **Control.Monad.Trans.Reader** module that comes with the **transformers** package. 

We will learn about Monad transformers in lesson 25.

You can also define the Reader monad without using a monad transformer that works equally as the one above. 

We will chose this aproach in this lesson. Let's first look at the definition of the **Reader** type:
```haskell
newtype Reader e a = Reader { runReader :: e -> a }
```

We see that the `Reader` data constructor holds a function that can be accessed with the name `runReader`.

It takes in a environment variable of type `e` and returns a variable of type `a`.

We can now create a Monad instance for `Reader e` and not just `Reader`.

This means the type of our environment variable will remain the same after we apply the `>>=` operator.
```haskell
(>>=) :: Reader e a        ->
         (a -> Reader e b) ->
         Reader e b
```

The monad instance for `Reader e` is then defined as:
```haskell
instance Monad (Reader e) where 
  return a = Reader $ const a
  readerMonad >>= f = Reader $ \env -> 
    let a = runReader readerMonad env
    in runReader (f a) env
```

We also need to define the Functor and Applicative instances.
```haskell
instance Functor (Reader e) where
    fmap f readerFunctor = Reader $ \env ->
        f $ runReader readerFunctor env

instance Applicative (Reader e) where
    pure a = Reader $ const a
    f <*> g = Reader $ \env ->
        let h = runReader f env
        in h $ runReader g env
```

### Helper functions

The **Control.Monad.Trans.Reader** module defines also some helper functions that work with the Reader moand. 

The `ask` function retrieves the value from the enviroment variable.
```haskell
ask :: Reader e e 
ask = Reader id
```

It is used inside a Reader Monad. To get the environment variable you also need to use the `<-` oprator that takes variables out of the monadic context.
```haskell
env <- ask
```

The `reader` function is the inverse operation of the `runReader` function. It takes in a function of type `e -> a` and produces a Reader monad `Reader e a`.
```haskell
reader :: (e -> a) -> Reader e a
reader f = do
    env <- ask
    return (f env)
```

The `local` function takes in a function and a Reader moand and updates the Reader monad by applying the function to the environment variable.
```haskell
local :: (e -> e) -> Reader e a -> Reader e a
local f readerM = Reader $ runReader readerM . f
```

It can also be used inside inside a Reader monad. Same as for `ask` function if you want to get the modified enviroment variable you need to use the `<-` operator. 

Another helper function is the `asks` function which performs the inverse operation of the `runReader` function.

It takes in a function of type `e -> a` and produces a Reader monad.
```haskell
asks :: (e -> a) -> Reader e a
asks f = do
    env <- ask
    return (f env)
```

The `asks` function is used inside a Reader to update the environment variable by applying the input function to it. 

### MonadReader type class

The Reader monad and the helper functions presented in the previous two chapters can be also found in the **Control.Monad.Reader** module.

This module is part of the **mtl** package that is a collection of monad classes, extending the **transformers** package.

The module defines the `MonadReader` type class that contains the functions `ask`, `local` and `reader`.

They are equivalent to the functions describer in the previous chapter.

```haskell
class Monad m => MonadReader e m | m -> e where
    ask   :: m e
    ask = reader id

    local :: (e -> e) -> m a -> m a

    reader :: (e -> a) -> m a
    reader f = do
      env <- ask
      return (f env)
    {-# MINIMAL (ask | reader), local #-}
```

The minimal complete definition is the `local` function and then either the `ask` or the `reader` function.

To be able to declare a type class parameterized with two types you need to use the *MultiParamTypeClasses* language pragma. 

If the type `m` which in reality for us will be `(Reader e)`, is defiinng the type `e` we can state this dependency with the `| m -> e` statement. 

To be able to use such dependency statements in type class declarations we need to include the *FunctionalDependencies* language pragma. 

For the `Reader e` monad the instance of the MonadReader type class would be:
```haskell
instance MonadReader e (Reader e) where
    ask :: Reader e e
    ask = Reader id
    
    local :: (e -> e) -> Reader e a -> Reader e a
    local f readerM = Reader $ runReader readerM . f
```

To be able to write such an instance we would also need to include the language pragmas *FlexibleInstances* and *InstanceSigs*.

The first let's you work with concrete types when creating an type class instances as e.g. if you would want to create an instance for `Maybe Int` instead of `Maybe a`.

The second enables to also write out type signatures as part of typeclass instance declarations.

### Simple example

Below is a simple Reader monad example where we use the `ask` and `runReader` functions.

The programm asks the user for his age and prints out two messages with retirement information for the user.

In [ ]:
import Control.Monad.Reader

ukInfo :: Reader Int String
ukInfo = do
    age <- ask
    let yearsToRetire = 66 - age
        msg = if yearsToRetire <= 0
              then "You can retire in the United Kingdom."
              else "In the United Kingdom you have to work up to " ++ show yearsToRetire ++ " more years." 
    return msg

thInfo :: Reader Int String
thInfo = do
    age <- ask
    let yearsToRetire = 60 - age
        msg = if yearsToRetire <= 0
              then "You can retire in Thailand."
              else "In the Thailand you have to work up to " ++ show yearsToRetire ++ " more years." 
    return msg

processAge :: Reader Int String
processAge = do
    ukMsg <- ukInfo
    thMsg <- thInfo
    let infoMsg = "The retirement information for your age:\n" ++
                  ukMsg ++ "\n" ++ thMsg
    return infoMsg

main :: IO ()
main = do
    putStrLn "Input your age:"
    age <- read <$> getLine
    putStrLn $ runReader processAge age

main

When we call the `ukInformation` and `thailandInformation` Reader monads inside the `processAge` Reader monad, the environment variabe of the `processAge` Reader monad is passed onto them.

## Reader monad examples

In our first example we update the code from the previous chapter such that we use the `local` function.

We modily the environment variable of the `ukInfo` and `thInfo` Reader monads by applying the function `add2Years` that icreases the age for 2.

In [ ]:
processAge :: Reader Int String
processAge = do
    ukMsg <- local add2Years ukInfo
    thMsg <- local add2Years thInfo
    let infoMsg = "The retirement information for your age in 2 years:\n" ++
                  ukMsg ++ "\n" ++ thMsg
    return infoMsg

add2Years :: Int -> Int
add2Years years = years + 2

main

We see that the environment variable from the `processAge` Reader monad gets passed onto the other two Reader monads after the function `add2Years` is applied to it.

Below is another example on how to lookup variables if you have them stored in a list. It is taken and adapted from [HaskellWiki](https://wiki.haskell.org/All_About_Monads#The_Reader_monad).

The list contains tuples where the first part is the variable name and the second is variable either of type `String` or of type `Template`. 

This example shows that you can also define a function that takes in data and returns a reader monad. It also uses the `asks` function.

In [ ]:
data Template = S String | V Template | T Template deriving Show

data Environment = Env {templates::[(String,Template)],
                        variables::[(String,String)]}

-- lookup a variable from the environment
lookupVar :: String -> Environment -> Maybe String
lookupVar name env = lookup name (variables env)

-- lookup a template from the environment
lookupTemplate :: String -> Environment -> Maybe Template
lookupTemplate name env = lookup name (templates env)

-- resolve a template into a string
resolve :: Template -> Reader Environment (String)
resolve (S s)    = return s
resolve (V t)    = do varName  <- resolve t
                      varValue <- asks (lookupVar varName)
                      return $ maybe "" id varValue
resolve (T t)    = do tmplName <- resolve t
                      body     <- asks (lookupTemplate tmplName)
                      return $ maybe "" show body

template1 :: Template
template1 = V (S "varName1")

template2 :: Template
template2 = T (S "varName2")

myEnv :: Environment
myEnv = Env {
      templates = [("varName1",template1),("varName2",template2)],
      variables = [("varName1","value1"),("varName2","value2")]
    }

main :: IO ()
main = do
    print $ runReader (resolve template1) myEnv
    print $ runReader (resolve template2) myEnv

main

In our last example let's implement now the code from the first chapter with the use of the **Reader monad**.

In [ ]:
processEnv :: Reader Environment Content
processEnv = do
    conf1 <- func1
    conf2 <- func2
    conf3 <- func3
    conf4 <- func4
    let conf = conf1 ++ "\n\n" ++
               conf2 ++ "\n\n" ++
               conf3 ++ "\n\n" ++
               conf4 ++ "\n\n"
    return conf
  
func1 :: Reader Environment Content
func1 = do
    env <- ask
    let parameter = fromJust $ Map.lookup "PATH" $ parameters env
        content = "System path includes: \n" ++ parameter
    return content

func2 :: Reader Environment Content
func2 = do
    env <- ask
    let parameter = fromJust $ Map.lookup "SHELL" $ parameters env
        content = "Shell used is: \n" ++ parameter
    return content

func3 :: Reader Environment Content
func3 = do
    env <- ask
    let parameter = fromJust $ Map.lookup "HOSTNAME" $ parameters env
        content = "Hostname is: \n" ++ parameter
    return content

func4 :: Reader Environment Content
func4 = do
    env <- ask
    let parameter = fromJust $ Map.lookup "HOME" $ parameters env
        content = "Current home directory is: \n" ++ parameter
    return content

main :: IO ()
main = do
    env <- loadEnv
    let configuration = runReader processEnv env
        pathParam = fromJust $ Map.lookup "HOME" $ parameters env
        filePath = pathParam ++ "/conf.txt"
    print "Writing configuration to file: conf.txt"
    writeFile filePath configuration

main

The `env` variable is passed from the `processEnv` function to all the other Reader monad functions.

Benefits compared to initial code: 
- you don't have to take care of passing around the environment data

- the Reader monads `func1` to `func4` have a cleaner type signature as the functions in the initial example.

## Recap

In this lesson we've discussed:

- the motivation for introducing the Reader monad type 

- the definition of the Reader monad type

- helper functions that work with the Reader monad

- examples that use the Reader monad type and its helper functions